Modelos GPT y estrategias de generación

Notebook de apoyo con **2 ejercicios en Python** para hacer durante la clase:

1. Laboratorio de **estrategias de decodificación** con un GPT pequeño.
2. Laboratorio de **ablación de contexto en prompts**.

Todo usando modelos open source (`distilgpt2`).

In [ ]:
!pip install -q transformers torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Ejercicio 1 – Estrategias de decodificación con un GPT pequeño

**Objetivo:** ver en la práctica cómo cambian las respuestas al usar greedy, sampling, distintas temperaturas, top-k y top-p.

**Requisitos:**
- `transformers`
- `torch`.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = 'distilgpt2'  # pequeño y razonablemente rápido
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Configurar pad_token (distilgpt2 no tiene uno por defecto)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


In [15]:
def generate_text(prompt, **gen_kwargs):
    inputs = tokenizer(prompt, return_tensors='pt')
    # Añadir parámetros por defecto para evitar repetición excesiva
    default_kwargs = {
        'pad_token_id': tokenizer.eos_token_id,
        'repetition_penalty': 1.1,  # Penaliza repetición
    }
    # Combinar con kwargs proporcionados (los kwargs del usuario tienen prioridad)
    gen_kwargs = {**default_kwargs, **gen_kwargs}
    with torch.no_grad():
        out_ids = model.generate(**inputs, **gen_kwargs)
    return tokenizer.decode(out_ids[0], skip_special_tokens=True)


### 1.1. Comparación básica

Probamos la misma frase inicial con varias configuraciones.

In [16]:
prompt = 'In a distant future, humans and AI'
print('PROMPT:\n', prompt)
print('\n=== Greedy (do_sample=False) ===')
print(generate_text(prompt, max_new_tokens=80, do_sample=False))
print('\n=== Sampling: temp=1.0, top_k=50 ===')
print(generate_text(prompt, max_new_tokens=80, do_sample=True, temperature=1.0, top_k=50))
print('\n=== Sampling: temp=0.3, top_p=0.9 ===')
print(generate_text(prompt, max_new_tokens=80, do_sample=True, temperature=0.3, top_p=0.9))


PROMPT:
 In a distant future, humans and AI

=== Greedy (do_sample=False) ===
In a distant future, humans and AI will be able to create new worlds.
The first step is to build the world of our own robots that can interact with each other in real time using their brains. The next stage involves building an artificial intelligence system called "AI" which could then work on any kind or form of human interaction between them. This would allow for more complex interactions such as social networking (like Facebook) where people are

=== Sampling: temp=1.0, top_k=50 ===
In a distant future, humans and AI will still share space with other homonin species. But this is where the question comes into play (though they do exist)—and if we get our hands on new ones to help humankind better understand their environment—it would likely be useful to find out what kinds of interactions you might have in common: how much time each person has spent communicating or when talking with others over an open In

### 1.2. Actividad

**Nota:** `distilgpt2` está entrenado principalmente en inglés. Para prompts en español, considera usar un modelo multilingüe como `gpt2` con fine-tuning o modelos como `mGPT`.

1. Cambia el `prompt` por uno diferente en inglés (historia, explicación técnica, etc.).
2. Prueba temperaturas 0.1, 0.7 y 1.5 manteniendo el resto igual.
3. Cambia `top_k` y `top_p` y comenta qué configuraciones dan textos más coherentes vs más creativos.
4. Experimenta con `repetition_penalty` (valores entre 1.0 y 1.5) para controlar la repetición.
5. Relaciona lo que observas con la teoría de decodificación vista en clase.

---

## Ejercicio 2 – Ablación de contexto en prompts

**Objetivo:** ver cómo pequeñas modificaciones en el prompt (quitar instrucciones) cambian el comportamiento del modelo.

Usaremos el mismo modelo `distilgpt2`.

In [17]:
def gen(prompt):
    print('PROMPT:\n', prompt)
    print('\nRESPUESTA:\n')
    print(generate_text(prompt, max_new_tokens=500, do_sample=True, temperature=0.7, top_p=0.9))


### 2.1. Prompt completo

**Nota importante:** `distilgpt2` está entrenado principalmente en inglés, por lo que los prompts en español pueden no funcionar bien. Este ejercicio muestra cómo el modelo responde a diferentes estructuras de prompt, incluso cuando el idioma no es óptimo.

In [21]:
# Ejemplo con prompt en inglés (funciona mejor con distilgpt2)
prompt_full = (
    'Answer in a professional tone, and summarize at the end in three bullet points. '
    'Topic: advantages of using Git in generative AI projects.'
)
gen(prompt_full)


PROMPT:
 Answer in a professional tone, and summarize at the end in three bullet points. Topic: advantages of using Git in generative AI projects.

RESPUESTA:

Answer in a professional tone, and summarize at the end in three bullet points. Topic: advantages of using Git in generative AI projects.
The following is an overview of how to use git for our project development purposes (i.e., from start-up). The topics will focus on some important things that need to be covered first as well as your own internal workings. For example, if you want to create new users or add their favorite apps just like we did with GitHub's ‪ Community‬s . You can also find other useful resources available here [1]. In general, this topic should not only cover advanced features such as support for shared libraries and tools but also about user experience — so please read these notes before discussing them!


### 2.2. Variantes del prompt

Ahora iremos quitando partes de la instrucción.

In [20]:
# Variantes del prompt en inglés (funciona mejor con distilgpt2)
prompts = {
    'without_bullets': 'Answer in a professional tone. Topic: advantages of using Git in generative AI projects.',
    'without_tone': 'Answer, and summarize at the end in three bullet points. Topic: advantages of using Git in generative AI projects.',
    'topic_only': 'Advantages of using Git in generative AI projects.',
}

for name, p in prompts.items():
    print('\n\n=== Variant:', name, '===\n')
    gen(p)




=== Variant: without_bullets ===

PROMPT:
 Answer in a professional tone. Topic: advantages of using Git in generative AI projects.

RESPUESTA:

Answer in a professional tone. Topic: advantages of using Git in generative AI projects.
I’m not going to say that this is an absolute good idea, but I think it will be useful if you have questions about the nature and usefulness of git libraries or how they are used by developers on StackOverflow (and other forums).


=== Variant: without_tone ===

PROMPT:
 Answer, and summarize at the end in three bullet points. Topic: advantages of using Git in generative AI projects.

RESPUESTA:

Answer, and summarize at the end in three bullet points. Topic: advantages of using Git in generative AI projects.
There are several issues that can be addressed here; one is it's not possible to use git as a whole without having multiple branches (for example, you have your own branch). This may cause some developers or programmers to switch back into different

### 2.3. Actividad

1. Cambia el tema (por ejemplo: *ethical risks of generative AI*, *MLOps best practices*, etc.).
2. Añade nuevas instrucciones (por ejemplo: *avoid technical jargon*, *use concrete examples*, *respond in JSON format*).
3. Observa qué instrucciones respeta mejor el modelo y cuáles ignora más a menudo.
4. Prueba diferentes estructuras de prompt (preguntas directas, instrucciones explícitas, ejemplos few-shot).
5. Piensa cómo diseñar prompts **más robustos** a partir de lo que ves.

**Nota:** Si quieres trabajar con español, considera usar un modelo multilingüe o fine-tuneado en español.